In [1]:
import pandas as pd

In [2]:
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)

In [3]:
df = pd.read_csv(r'parsed_routes_data\all_routes_data.csv') #real dataset
#df = pd.read_csv(r'parsed_routes_data\first_bulk_of_routes_data.csv') # small sample dataset for testing
df.rename(columns={'Unnamed: 0': 'route_id', 'Unnamed: 1': 'num_of_waypoint'}, inplace=True)
df

,route_id,num_of_waypoint,lat_lgt,latitude,longitude
0,1000303.gpx,0,"('38.29842', '-2.65105')",38.29842,-2.65105
1,1000303.gpx,1,"('38.29647', '-2.65079')",38.29647,-2.65079
2,1000303.gpx,2,"('38.29418', '-2.65011')",38.29418,-2.65011
3,1000303.gpx,3,"('38.29499', '-2.65174')",38.29499,-2.65174
4,1000303.gpx,4,"('38.29674', '-2.65174')",38.29674,-2.65174
...,...,...,...,...,...
14486915,99997.gpx,60,"('48.27417', '16.33581')",48.27417,16.33581
14486916,99997.gpx,61,"('48.27445', '16.33493')",48.27445,16.33493
14486917,99997.gpx,62,"('48.27461', '16.33503')",48.27461,16.33503
14486918,99997.gpx,63,"('48.27464', '16.33575')",48.27464,16.33575


In [4]:
#converting str values from 'routes_id' (that is going to become an index) into ints
df['route_id'] = df['route_id'].str.split('.').str[0].astype(int)

#converting str values from 'num_of_waypoint' (that is going to become column names) into ints
df['num_of_waypoint'] = df['num_of_waypoint'].astype(int)

In [5]:
#reshaping df from long format to wide format

#pip install pandas==0.21
#import pandas as pd
df_reshaped = df.pivot(index='route_id', columns='num_of_waypoint', values='lat_lgt')
#pip install pandas

df_reshaped

ValueError: Unstacked DataFrame is too big, causing int32 overflow

In [ ]:
#exploring the new shape
df_reshaped.shape # --> (1830, 29852) (smaller size sample)

#since the df now has 29852 columns, I want to verify, if the longest route really has 29852 waypoints or if something went wrong
df['num_of_waypoint'].max()

#finding out which route has that many waypoints, because I want to have a look into the original gps file
df.sort_values(by = 'num_of_waypoint', ascending = False) #--> it's 2825871.gpx (that hypercorrect documentation of a file should be excluded)

#exploring the distribution of the total number of waypoints per route a bit more
routes_grp = df.groupby(['route_id'])
routes_grp['num_of_waypoint'].max().median() #--> median of total waypoints per route is 252 (smaller size sample)
routes_grp['num_of_waypoint'].max().describe() #--> 75% is at 507 total waypoints: by including 1000 columns I will cover most of the routes and exclude outliers

#dropping rows with more than 1000 waypoints from df_reshaped
df_reshaped[1000].isna().value_counts() #verifing not too many routes would be deteled
df_reshaped = df_reshaped[df_reshaped[1000].isna()]

#dropping every column with only NaN values in it (= columns 1000 and higher)
df_reshaped.isnull().sum(axis=0).value_counts() #verifing that it's column 0-999 that stay
df_reshaped = df_reshaped.dropna(axis=1, how='all')
df_reshaped.shape

(1698, 1000)

In [ ]:
df_reshaped
df_reshaped.to_csv('cl1_all_routes_data.csv')